In [3]:
suppressPackageStartupMessages({
  library(dplyr)
  library(pagoda2)
  library(cowplot)
  library(magrittr)
  library(dataorganizer)

  devtools::load_all()
})

ℹ Loading cellAdmixNotebooks
Warning message:
“Objects listed as exports, but not present in namespace:
• plot_expression_comparison”


In [4]:
sos <- prepare_BC_sc_spatial(load_molecules=FALSE)
so_rna <- sos$so_rna
so_spatial <- sos$so_spatial

In [7]:
# make pagoda objects for the sc and spatial datasets
sc_counts <- so_rna[['RNA']]$counts
spatial_counts <- so_spatial[['RNA']]$counts
spatial_counts <- spatial_counts[rownames(sc_counts),]

sc_obj <- Pagoda2$new(sc_counts,log.scale=TRUE, n.cores=20)
spatial_obj <- Pagoda2$new(spatial_counts,log.scale=TRUE, n.cores=20)

Warning message in LayerData.Assay5(object = x, layer = i):
“multiple layers are identified by counts.Gene Expression counts.Blank Codeword counts.Negative Control Codeword counts.Negative Control Probe
 only the first layer is used”


21847 cells, 313 genes; normalizing ... 

Using plain model 

log scale ... 

done.


158030 cells, 313 genes; normalizing ... 

Using plain model 

log scale ... 

done.




In [8]:
# run DE
ct_annot <- factor(so_rna@meta.data$cell_type,levels=unique(so_rna@meta.data$cell_type))
names(ct_annot) <- rownames(so_rna@meta.data)
de_out_mark <- sc_obj$getDifferentialGenes(
  verbose=TRUE,groups=ct_annot,z.threshold = 0,
  upregulated.only=TRUE,append.auc = TRUE
)

# put into a table
de_genes <- lapply(1:length(de_out_mark),function(i){
  ct_nm <- names(de_out_mark)[i]
  x <- de_out_mark[[i]]

  # add pvalues
  x$padj <- 2*pnorm(abs(x$Z), mean = 0, sd = 1, lower.tail = FALSE)

  x$cell_type <- ct_nm
  rownames(x) <- NULL
  return(x)
})

running differential expression with 18 clusters ... 

adjusting p-values ... 

done.




In [9]:
de_genes_full1 <- do.call("rbind.data.frame", de_genes)

# subset to same genes in spatial data
de_genes_full1 <- de_genes_full1[de_genes_full1$Gene %in% rownames(so_spatial),]

In [10]:
# subset to significant results only
de_genes_full1 <- de_genes_full1[de_genes_full1$padj<.05,]
de_genes_full1 <- de_genes_full1[de_genes_full1$AUC>.55,]
de_genes_full1 <- de_genes_full1[de_genes_full1$Specificity>.95,]

# order by AUC
de_genes_full1 <- de_genes_full1[order(de_genes_full1$AUC,decreasing = TRUE),]

ct_list <- lapply(unique(de_genes_full1$cell_type),function(ct){
  return(de_genes_full1[de_genes_full1$cell_type==ct,])
})
names(ct_list) <- unique(de_genes_full1$cell_type)

markers_plot <- unique(de_genes_full1$Gene)
all_expr_counts <- list()
for (ct in unique(so_rna@meta.data$cell_type)) {
  cells_keep <- rownames(so_rna@meta.data)[as.character(so_rna@meta.data$cell_type)==ct]
  so_rna_sub <- subset(so_rna,cells=cells_keep)
  expr <- sc_counts[markers_plot,]
  cell_expr_counts <- rowSums(expr>0) / ncol(expr)
  all_expr_counts[[ct]] <- cell_expr_counts
}

all_expr_counts <- do.call(cbind,all_expr_counts)

for (ct in unique(de_genes_full1$cell_type)) {
  marker_df_sub <- de_genes_full1[de_genes_full1$cell_type==ct,]
  potential_markers <- marker_df_sub$Gene

  # now remove the markers if they are more highly expressed in other cell types
  g_rem_all <- c()
  for (mark in potential_markers) {
    de_sub_efrac <- all_expr_counts[mark,]
    de_sub_efrac2_other <- de_sub_efrac[names(de_sub_efrac)!=ct]
    if (de_sub_efrac[ct]<max(de_sub_efrac2_other)) {
      g_rem_all <- c(g_rem_all,mark)
    }

    if (sum(de_sub_efrac2_other>.6)>1) {
      g_rem_all <- c(g_rem_all,mark)
    }

  }
  ct_dat <- ct_list[[ct]]
  ct_dat <- ct_dat[!(ct_dat$Gene %in% g_rem_all),]
  ct_list[[ct]] <- ct_dat
}

# extract markers to plot
markers_plot <- lapply(unique(so_rna@meta.data$cell_type),function(ct) {
  marker_df_sub <- ct_list[[ct]]
  if (length(marker_df_sub$Gene) > 5) {
    return(marker_df_sub$Gene[1:5])
  } else {
    return(marker_df_sub$Gene)
  }
})
markers_plot <- unique(unlist(markers_plot))
markers_plot <- rev(markers_plot)
markers_plot

[1] "TIMP4"    "LEP"      "LPL"      "ADH1B"    "ADIPOQ"   "RAB30"   
 [7] "TNFRSF17" "DERL3"    "PIGR"     "ALDH1A3"  "KRT23"    "SLAMF7"  
[13] "CD274"    "CD80"     "FAM49A"   "CTSG"     "KIT"      "HDC"     
[19] "TPSAB1"   "CPA3"     "C5orf46"  "KRT6B"    "ACTG2"    "MYH11"   
[25] "SPIB"     "LILRA4"   "IL3RA"    "PLD4"     "GZMB"     "CD19"    
[31] "CD79B"    "CD79A"    "BANK1"    "MS4A1"    "CAVIN2"   "RAMP2"   
[37] "CLEC14A"  "VWF"      "LY86"     "IL2RA"    "MRC1"     "KLRD1"   
[43] "CD247"    "CD3G"     "GZMA"     "CD8A"     "LGALSL"   "HPX"     
[49] "RAPGEF3"  "STC1"     "HMGA1"    "DPT"      "SFRP4"    "IGF1"    
[55] "PDGFRA"   "PDGFRB"   "KDR"      "EDNRB"    "AVPR1A"   "FBLIM1"  
[61] "NDUFA4L2" "CD86"     "APOC1"    "MNDA"     "FCGR3A"   "HAVCR2"

In [11]:
# save the full table of thresholded markers
ct_list_df <- do.call(rbind.data.frame, ct_list)
write.csv(ct_list_df, file=CachePath('BC_markers_sub.csv'))